In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17320 (from 3)
Receiving objects: 100% (17410/17410), 16.31 MiB | 29.61 MiB/s, done.
Resolving deltas: 100% (11929/11929), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta

In [2]:
import torch
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large")  # or "DPT_Hybrid" for faster
midas.to('cuda').eval()
#midas.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.l

In [3]:
import numpy as np
from pathlib import Path
from google.colab import files
import cv2

In [4]:
# Load YOLOv5 model
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to('cuda')
#model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-4-18 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 124MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [14]:
import numpy as np
import cv2
import torch
import json
from tqdm import tqdm

# Determine device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Specify the video file path
video_path = "/kaggle/input/video-test/WhatsApp Video 2025-04-11 at 19.04.37_c67eedf2.mp4"

# Open the video
cap = cv2.VideoCapture(video_path)

# Sanity check: did it open?
if not cap.isOpened():
    print("❌ Could not open the video.")
else:
    print("✅ Video opened successfully.")

frame_count = 0
frame_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get the total number of frames in the video

# Get video properties for output
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer for output
output_video_path = "/kaggle/working/output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# List to store detection information for all processed frames
all_detections = []

# Move MiDaS and YOLO models to the appropriate device
midas = midas.to(device)
model_yolo = model_yolo.to(device)

# Initialize the tqdm progress bar
with tqdm(total=frame_total, desc="Processing Frames", unit="frame") as pbar:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process every frame
        original_frame = frame.copy()
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run MiDaS depth estimation
        input_batch = transform(img_rgb).to(device)  # Move input to the same device
        with torch.no_grad():
            prediction = midas(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img_rgb.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()
            depth_map = prediction.cpu().numpy()  # Move back to CPU for numpy

        # Normalize depth map to range 0-255 for color mapping
        depth_map_normalized = np.uint8(np.clip((depth_map - np.min(depth_map)) / (np.max(depth_map) - np.min(depth_map)) * 255, 0, 255))

        # Apply colormap to depth map
        depth_map_colored = cv2.applyColorMap(depth_map_normalized, cv2.COLORMAP_JET)

        # Run YOLO detection
        results = model_yolo(original_frame)
        detections = results.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2, conf, class]

        # List to store detections for the current frame
        frame_detections = []

        # Process each detection
        for det in detections:
            x1, y1, x2, y2, conf, cls = map(int, det[:6])
            label = model_yolo.names[cls]

            # Estimate depth inside the bbox
            depth_crop = depth_map[y1:y2, x1:x2]
            if depth_crop.size > 0:
                avg_depth = np.median(depth_crop)
            else:
                avg_depth = 0

            # Draw box and depth label
            cv2.rectangle(original_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(original_frame, f"{label} {avg_depth:.2f}m", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Store detection information: [label, bottom-left (x1, y2), bottom-right (x2, y2)]
            detection_info = [label, [x1, y2], [x2, y2]]  # Using lists for JSON compatibility
            frame_detections.append(detection_info)

        # Add frame detections to the overall list
        all_detections.append({
            'frame': frame_count,
            'detections': frame_detections
        })

        # Combine the original frame with the colored depth map for visualization
        combined_frame = cv2.addWeighted(original_frame, 0.7, depth_map_colored, 0.3, 0)

        # Write frame to output video
        out.write(combined_frame)

        frame_count += 1
        pbar.update(1)  # Update the progress bar after processing each frame

        # Stop after processing a certain number of frames (optional)
        if frame_count > 10000:
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Save all detections to a JSON file
output_file = "/kaggle/working/detections.json"
with open(output_file, 'w') as f:
    json.dump(all_detections, f, indent=4)

print(f"Detections saved to {output_file}")
print(f"Output video saved to {output_video_path}")

Using device: cuda
✅ Video opened successfully.


Processing Frames:   0%|          | 0/1541 [00:00<?, ?frame/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing Frames:   0%|          | 1/1541 [00:00<10:55,  2.35frame/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing Frames:   0%|          | 2/1541 [00:00<09:59,  2.57frame/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing Frames:   0%|          | 3/1541 [00:01<09:42,  2.64frame/s]/root/.cache/torch/hub/ultralytics_yolov5_master/mode

Detections saved to /kaggle/working/detections.json
Output video saved to /kaggle/working/output.mp4
